In [35]:
import os
import json
import re

import pandas as pd
import numpy as np

import redis
from redis.commands.search.field import VectorField, TextField
from redis.commands.search.query import Query
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.field import TagField

from openai import OpenAI

In [8]:
os.environ['OPENAI_API_KEY'] = ''

In [9]:
client = OpenAI()

In [10]:
# Vectorize the query using OpenAI's text-embedding-ada-002 model
def get_embedding(text):
    # vectorize with OpenAI text-emebdding-ada-002
    embedding = client.embeddings.create(
        input=text,
        model="text-embedding-ada-002")

    vector = embedding.data[0].embedding
    
    return vector

# JSON y esquema

In [3]:
terminos = None

with open('terminos.json') as f:
    terminos = json.load(f)

print(len(terminos))
print(terminos['9']['tag'])

16
repatriacion


In [4]:
REDIS_HOST = '127.0.0.1'
REDIS_PORT = 6379
REDIS_PASSWORD = ''

schema = [
    TagField("tag"),
    TextField("titulo"), 
    TextField("description"),
    TextField("contenido"),
    VectorField("embedding", "HNSW",
                {"TYPE": "FLOAT32",
                 "DIM": 1536,
                 "DISTANCE_METRIC": "COSINE"}),
   ]

In [5]:
conn = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, password=REDIS_PASSWORD, encoding='utf-8', decode_responses=True)

In [6]:
for i in range(0, len(terminos)):

    termino = terminos[str(i)]

    tag = termino['tag']
    titulo = termino['titulo']
    descripcion = termino['descripcion']

    for j in range(0, len(termino['embedding'])):
          
        redis_hash = 'termino:' + str(i) + '_' + str(j)
        vector = np.array(termino['embedding'][j]).astype(np.float32).tobytes()
        
        mapping = {'tag': tag, 
                   'titulo': titulo,
                   'descripcion': descripcion,
                   'contenido': termino['contenido'][j],
                   'embedding': vector}
        

        respuesta = conn.hset(redis_hash, mapping=mapping)
    print(i)    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [7]:
conn.ft('terminos').create_index(fields=schema, definition=IndexDefinition(prefix=["termino:"], index_type=IndexType.HASH))

'OK'

# Busqueda en REDIS

In [20]:
def hybrid_search(query_vector, client_redis, top_k=3, hybrid_fields="*"):
    
    base_query = f"{hybrid_fields}=>[KNN {top_k} @embedding $vector AS vector_score]"
    query = Query(base_query).return_fields(
        "tag",
        "titulo",
        "descripcion",
        "contenido",
        "vector_score").sort_by("vector_score").dialect(2)

    results = client_redis.ft("terminos").search(query, query_params={"vector": query_vector})

    return results

In [32]:
busquedas = ['deportes', 'cobertura por accidente', 'repatriacion sanitaria', 
             'repatriacion_funeraria', 'cancelacion de viaje', 'excepciones', 
             'transferencia de fondos', 'asesoria legal', 'montañismo', 'hospitalizacion']

In [33]:
array_termino_buscado = []
array_titulo_encontrado = []
array_contenido_encontrado = []

for busqueda in busquedas:

    print('Procesando ', busqueda)

    #VECTORIZAR
    busqueda_vector = get_embedding(busqueda)
    busqueda_bytes = np.array(busqueda_vector).astype(np.float32).tobytes()
    
    #BUSQUEDA EN REDIS
    resultados = hybrid_search(busqueda_bytes, conn)
    
    if resultados:
        #print(f"Found {resultados.total} results:")
        for i, termino in enumerate(resultados.docs):
            score = 1 - float(termino.vector_score)
    
            if score < 0.7:
                continue
                
            #print('\n', termino.contenido)
            #print('')
            #print(i, termino.titulo, ' --- ', termino.tag, round(score ,3))
            #print(termino.descripcion) 
            #print('\n***********************************************************\n')
    
            array_termino_buscado.append(busqueda)
            array_titulo_encontrado.append(termino.titulo)
            #array_contenido_encontrado.append(termino.contenido)
    else:
        print("No results found")

Procesando  deportes
Procesando  cobertura por accidente
Procesando  repatriacion sanitaria
Procesando  repatriacion_funeraria
Procesando  cancelacion de viaje
Procesando  excepciones
Procesando  transferencia de fondos
Procesando  asesoria legal
Procesando  montañismo
Procesando  hospitalizacion


In [46]:
resultados_busquedas = pd.DataFrame({'termino': array_termino_buscado, 'titulo': array_titulo_encontrado})
resultados_busquedas = resultados_busquedas.drop_duplicates()
resultados_busquedas = resultados_busquedas.reset_index(drop=True)
print(resultados_busquedas.shape)
display(resultados_busquedas.head(20))

(20, 2)


,termino,titulo
0,deportes,Upgrade Deporte
1,deportes,Práctica recreativa de deportes
2,cobertura por accidente,Compensación por daños en el equipaje (tramos ...
3,cobertura por accidente,Asistencia Legal
4,cobertura por accidente,Regreso anticipado por siniestro en el domicilio
5,repatriacion sanitaria,Repatriaciones Sanitarias
6,repatriacion_funeraria,Repatriación o Traslado de restos
7,repatriacion_funeraria,Repatriaciones Sanitarias
8,cancelacion de viaje,Reintegro de gastos por imposibilidad de embarque
9,cancelacion de viaje,Asistencia Médica por enfermedad o accidente


In [47]:
output_json = resultados_busquedas.to_dict(orient='index')

with open('resultados_busquedas.json', 'w') as f:
    json.dump(output_json, f)

# Prompt

In [48]:
inputs = None

with open('resultados_busquedas.json') as f:
    inputs = json.load(f)

print(len(inputs))

20


In [51]:
prompt_general = ''
for i in range(0, len(terminos)):
    c_titulo = terminos[str(i)]['titulo']
    c_descripcion = terminos[str(i)]['descripcion']

    prompt_parcial = ''

    for j in range(0, len(inputs)):
        if inputs[str(j)]['titulo'] == c_titulo:
            prompt_parcial = prompt_parcial + inputs[str(j)]['termino'] + ', '

    prompt_general = prompt_general + '- Las busquedas de ' + prompt_parcial + 'tienen relacion con la clausula titulada '
    prompt_general = prompt_general + c_titulo + ' que dice ' + c_descripcion + '\n\n'

In [72]:
content_system = f"""Eres un asistente IA que ayuda a las personas a encontrar informacion
                    acerca de nuestra compañia de seguros y nada más. 
                    Una de tus tareas es proporcionar informacion de los terminos y condiciones en nuestros contratos.
                    La relacion entre busquedas y clausulas es la siguiente:
                    {prompt_general}"""

conversation = [{'role': 'system', 'content': content_system}]

In [73]:
#user_input = 'Hola, ¿podrias darme información acerca de cobertura de deportes?'
#user_input = 'Hola, ¿venden hamburguesas?'
#user_input = 'Hola, ¿cuentan con servicios de asesoria legal?'
#user_input = 'Hola, ¿podrias darme información acerca de los tipos de repatriaciones que manejan?'
user_input = 'Hola, ¿podrias darme información acerca de la cobertura medica?'

conversation.append({'role': 'user', 'content': user_input})

In [74]:
response = client.chat.completions.create(model='gpt-4o-mini-2024-07-18',
                                          messages=conversation)

In [75]:
bot_answer = response.choices[0].message.content
print(bot_answer)

La cobertura médica está relacionada con la cláusula titulada "Asistencia Médica por enfermedad o accidente". Según esta cláusula, los servicios de asistencia médica ofrecidos por la prestadora comprenden exclusivamente el tratamiento de urgencia del cuadro agudo que impida la continuación del viaje. Esta asistencia incluye:

a) Consultas médicas: La prestadora podrá coordinar visitas a domicilio siempre que cumplan con ciertas condiciones.

b) Teleasistencia médica: Proporciona información general sobre salud y puede incluir la emisión de una receta médica y/o medicamentos, según las leyes locales y el diagnóstico.

c) Atención por especialista: Si es necesario y autorizado por los equipos médicos de urgencia y la Central Operativa de la prestadora. 

Si necesitas información más específica o detalles adicionales, házmelo saber.
